In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# Explicit Waits 사용을 위한 라이브러리
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException

# 크롬 드라이버 자동 업데이트
from webdriver_manager.chrome import ChromeDriverManager

import time

In [2]:
# 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

In [3]:
# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [4]:
service = Service(executable_path=ChromeDriverManager().install())
service

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\Administrator\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [5]:
driver = webdriver.Chrome(service=service, options=chrome_options)
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="7dc538c70a2e81bc81eb8239e2bd51a1")>

In [6]:
# 웹페이지 해당 주소 이동
driver.implicitly_wait(5)  # 웹페이지가 로딩 될때까지 5초 기다림
driver.maximize_window()  # 화면 최대화

# 21대 국회의원 선거 스크래핑

In [7]:
# 시작. 페이지로 출발
driver.get('http://info.nec.go.kr/')

In [8]:
driver.switch_to.default_content()

In [9]:
driver.switch_to.frame('main')

In [10]:
driver.find_element(By.XPATH, '//*[@id="header"]/ul/li[2]/a/span[text() = "역대선거"]').click() # 역대선거
driver.implicitly_wait(5)

In [11]:
driver.find_element(By.XPATH, '//*[@id="presubmu"]/li[4]/a/span[text() = "투·개표"]').click() # 투개표
driver.implicitly_wait(5)

### 개표현황(읍면동별) 데이터프레임 처리 및 스크래핑한 자료 검증

- 콤보박스에서 조회할 목록(시도명, 구시군명)을 가져오고 스크랩한 데이터의 조회결과 내용과 같은지 비교한다.
- 매칭이 되지 않을 경우 프로그램 중단

In [12]:
# 개표현황(읍면동별)

def get_data(city, town):
    html = driver.page_source

    df1 = pd.read_html(html, header=0)
    df2 = df1[0].copy()
    
    result = driver.find_element(By.XPATH, '//*[@id="contentarea"]/div[2]/div[2]/div[1]/p[@class="selectCondition"]').text
    result = result.strip().replace('[', '').replace(']', '').replace('\n', ' ')
    
    if city in result:
        if town in result:
            df2['조회결과'] = result
        else:
            print(f'에러 - 조회결과에 문제가 있습니다.\n요청 : {city} {town}\n결과 : {result}')
            print('프로그램을 중단합니다.')
            sys.exit()
    else:
        print(f'에러 - 조회결과에 문제가 있습니다\n요청 : {city}\n결과 : {result}')
        print('프로그램을 중단합니다.')
        sys.exit()

    return df2

In [13]:
# 개표현황(읍면동별)

driver.find_element(By.XPATH, '//*[@id="header"]/div[4]/ul/li[6]/a[text() = "개표현황(읍면동별)"]').click()
driver.implicitly_wait(5)

driver.find_element(By.XPATH, '//*[@id="electionType2"]').click() # 국회의원 선거
time.sleep(1)

driver.find_element(By.XPATH, '//*[@id="electionName"]').send_keys('제21대')
time.sleep(1)

driver.find_element(By.XPATH, '//*[@id="electionCode"]').send_keys('국회의원선거')
time.sleep(1)

df_21 = pd.DataFrame()

combo_box_3 = driver.find_element(By.XPATH, '//*[@id="cityCode"]')
cityCodes = [option.text for option in combo_box_3.find_elements(By.TAG_NAME, "option")]
cityCodes.pop(0)

for cityCode in cityCodes:
    driver.find_element(By.XPATH, '//*[@id="cityCode"]').send_keys(cityCode)
    time.sleep(1)
    
    combo_box_4 = driver.find_element(By.XPATH, '//*[@id="townCode"]')
    townCodes = [option.text for option in combo_box_4.find_elements(By.TAG_NAME, "option")]
    townCodes.pop(0)    
    
    for townCode in townCodes:
        driver.find_element(By.XPATH, '//*[@id="townCode"]').send_keys(townCode)
        time.sleep(1)
        
        driver.find_element(By.XPATH, '//*[@id="searchBtn"]').click()
        driver.implicitly_wait(5)
        
        df_21 = pd.concat([df_21, get_data(cityCode, townCode)], ignore_index=True)
        time.sleep(1)
        
df_21.tail()

,선거구명,읍면동명,구분,선거인수,투표수,후보자별 득표수,후보자별 득표수.1,후보자별 득표수.2,후보자별 득표수.3,후보자별 득표수.4,후보자별 득표수.5,후보자별 득표수.6,후보자별 득표수.7,후보자별 득표수.8,후보자별 득표수.9,후보자별 득표수.10,후보자별 득표수.11,무효투표수,기권수,조회결과
12823,NaN,중문동,선거일투표,6478,3581,1641,1820,56,3517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64,2897,제21대 국회의원선거 국회의원선거 제주특별자치도 서귀포시
12824,NaN,예래동,계,3197,2155,917,1159,31,2107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,1042,제21대 국회의원선거 국회의원선거 제주특별자치도 서귀포시
12825,NaN,예래동,관내사전투표,765,764,346,393,6,745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,1,제21대 국회의원선거 국회의원선거 제주특별자치도 서귀포시
12826,NaN,예래동,선거일투표,2432,1391,571,766,25,1362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,1041,제21대 국회의원선거 국회의원선거 제주특별자치도 서귀포시
12827,NaN,잘못 투입·구분된 투표지,NaN,0,8,7,1,0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-8,제21대 국회의원선거 국회의원선거 제주특별자치도 서귀포시


## 21대 총선 데이터프레임 원본 (df_21)

In [14]:
# 선거구분, 시도명, 구시군명 열 분리

df_21['선거구분'] = df_21['조회결과'].str.split().str[2]
df_21['시도명'] = df_21['조회결과'].str.split().str[3]
df_21['구시군명'] = df_21['조회결과'].str.split().str[4]
df_21 = df_21[['선거구분', '시도명', '구시군명', '선거구명', '읍면동명', '구분', '선거인수', '투표수', '후보자별 득표수', '후보자별 득표수.1',
       '후보자별 득표수.2', '후보자별 득표수.3', '후보자별 득표수.4', '후보자별 득표수.5', '후보자별 득표수.6',
       '후보자별 득표수.7', '후보자별 득표수.8', '후보자별 득표수.9', '후보자별 득표수.10', '후보자별 득표수.11',
       '무효투표수', '기권수', '조회결과']]
df_21.shape

(12828, 23)

In [15]:
df_21 = df_21[df_21['후보자별 득표수'] != '후보자별 득표수'].copy() # 필요없는 값 삭제
df_21.shape

(12758, 23)

In [16]:
df_21['선거구명'] = df_21['선거구명'].shift(periods=-1) # 한칸 비틀어 올리고
df_21['선거구명'] = df_21['선거구명'].replace('선거구명', np.nan) # 필요없는 값은 NaN으로 변경
df_21['선거구명'] = df_21['선거구명'].fillna(method='ffill') # 결측값 연속 채우기
df_21.shape

(12758, 23)

### 21대 총선 후보자 목록 분리 (df_21_candi)

In [17]:
df_21_candi = df_21[df_21['후보자별 득표수'].str.contains('더불어민주당')]
df_21_candi.shape

(329, 23)

In [18]:
df_21_candi.tail()

,선거구분,시도명,구시군명,선거구명,읍면동명,구분,선거인수,투표수,후보자별 득표수,후보자별 득표수.1,...,후보자별 득표수.5,후보자별 득표수.6,후보자별 득표수.7,후보자별 득표수.8,후보자별 득표수.9,후보자별 득표수.10,후보자별 득표수.11,무효투표수,기권수,조회결과
12576,국회의원선거,경상남도,거창군,산청군함양군거창군합천군,읍면동명,구분,선거인수,투표수,더불어민주당서필상,미래통합당강석진,...,국가혁명배당금당김태영,무소속김태호,계,NaN,NaN,NaN,NaN,무효투표수,기권수,제21대 국회의원선거 국회의원선거 경상남도 거창군
12619,국회의원선거,경상남도,합천군,산청군함양군거창군합천군,읍면동명,구분,선거인수,투표수,더불어민주당서필상,미래통합당강석진,...,국가혁명배당금당김태영,무소속김태호,계,NaN,NaN,NaN,NaN,무효투표수,기권수,제21대 국회의원선거 국회의원선거 경상남도 합천군
12678,국회의원선거,제주특별자치도,제주시,제주시갑,NaN,NaN,NaN,NaN,더불어민주당송재호,미래통합당장성철,...,무소속박희수,무소속임효준,계,NaN,NaN,NaN,NaN,NaN,NaN,제21대 국회의원선거 국회의원선거 제주특별자치도 제주시
12727,국회의원선거,제주특별자치도,제주시,제주시을,NaN,NaN,NaN,NaN,더불어민주당오영훈,미래통합당부상일,...,NaN,NaN,계,NaN,NaN,NaN,NaN,NaN,NaN,제21대 국회의원선거 국회의원선거 제주특별자치도 제주시
12770,국회의원선거,제주특별자치도,서귀포시,서귀포시,읍면동명,구분,선거인수,투표수,더불어민주당위성곤,미래통합당강경필,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,무효투표수,기권수,제21대 국회의원선거 국회의원선거 제주특별자치도 서귀포시


In [19]:
df_21_candi = df_21_candi[['선거구분', '시도명', '구시군명', '선거구명', '후보자별 득표수',
                           '후보자별 득표수.1', '후보자별 득표수.2', '후보자별 득표수.3', '후보자별 득표수.4', '후보자별 득표수.5',
                           '후보자별 득표수.6', '후보자별 득표수.7', '후보자별 득표수.8', '후보자별 득표수.9', '후보자별 득표수.10',
                           '후보자별 득표수.11', '조회결과']]

In [20]:
df_21_candi.columns = ['선거구분', '시도명', '구시군명', '선거구명',
                       '후보자1', '후보자2', '후보자3', '후보자4', '후보자5', '후보자6', '후보자7', '후보자8',
                       '후보자9', '후보자10', '후보자11', '후보자12', '조회결과']

In [21]:
df_21_candi.loc[:,'후보자1' : '후보자12'] = df_21_candi.loc[:,'후보자1' : '후보자12'].replace('계', np.nan) # 쓰레기 값 삭제
df_21_candi.head()

,선거구분,시도명,구시군명,선거구명,후보자1,후보자2,후보자3,후보자4,후보자5,후보자6,후보자7,후보자8,후보자9,후보자10,후보자11,후보자12,조회결과
0,국회의원선거,서울특별시,종로구,종로구,더불어민주당이낙연,미래통합당황교안,우리공화당한민호,민중당오인환,가자!평화인권당이정희,공화당신동욱,국가혁명배당금당박준영,국민새정당백병찬,민중민주당박소현,한나라당김형석,무소속김용덕,NaN,제21대 국회의원선거 국회의원선거 서울특별시 종로구
58,국회의원선거,서울특별시,중구,중구성동구을,더불어민주당박성준,미래통합당지상욱,국가혁명배당금당이주양,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제21대 국회의원선거 국회의원선거 서울특별시 중구
122,국회의원선거,서울특별시,용산구,용산구,더불어민주당강태웅,미래통합당권영세,민생당권혁문,정의당정연욱,민중당김은희,국가혁명배당금당김희전,NaN,NaN,NaN,NaN,NaN,NaN,제21대 국회의원선거 국회의원선거 서울특별시 용산구
177,국회의원선거,서울특별시,성동구,중구성동구갑,더불어민주당홍익표,미래통합당진수희,정의당정혜연,국가혁명배당금당이정섭,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제21대 국회의원선거 국회의원선거 서울특별시 성동구
224,국회의원선거,서울특별시,광진구,광진구갑,더불어민주당전혜숙,미래통합당김병민,민생당임동순,정의당오봉석,국가혁명배당금당이승욱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제21대 국회의원선거 국회의원선거 서울특별시 광진구


#### 당명 공백으로 분리

- replace는 기본적으로 좌측부터 처리
- 이번 선거에서 당명은 '국가혁명배당금당'을 제외하면 '당'으로 끝나고 1자만 들어간다.
- '국가혁명배당금당'은 2번 처리
- '친박연대' 같은 당이 나온다면 추가로 처리해야 한다.
- replace 는 1회만 처리한다. ('당' 이 이름에 들어가 있는 경우도 있을 수 있음)

In [22]:
for i in range(4, 16):
    df_21_candi.iloc[:, i] = df_21_candi.iloc[:, i].str.replace('당', '당 ', 1)
    df_21_candi.iloc[:, i] = df_21_candi.iloc[:, i].str.replace('무소속', '무소속 ', 1)

for i in range(4, 16):
    df_21_candi.iloc[:, i] = df_21_candi.iloc[:, i].str.replace('배당 금당', '배당금당 ', 1)

df_21_candi = df_21_candi.reset_index(drop=True)
df_21_candi.head()

,선거구분,시도명,구시군명,선거구명,후보자1,후보자2,후보자3,후보자4,후보자5,후보자6,후보자7,후보자8,후보자9,후보자10,후보자11,후보자12,조회결과
0,국회의원선거,서울특별시,종로구,종로구,더불어민주당 이낙연,미래통합당 황교안,우리공화당 한민호,민중당 오인환,가자!평화인권당 이정희,공화당 신동욱,국가혁명배당금당 박준영,국민새정당 백병찬,민중민주당 박소현,한나라당 김형석,무소속 김용덕,NaN,제21대 국회의원선거 국회의원선거 서울특별시 종로구
1,국회의원선거,서울특별시,중구,중구성동구을,더불어민주당 박성준,미래통합당 지상욱,국가혁명배당금당 이주양,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제21대 국회의원선거 국회의원선거 서울특별시 중구
2,국회의원선거,서울특별시,용산구,용산구,더불어민주당 강태웅,미래통합당 권영세,민생당 권혁문,정의당 정연욱,민중당 김은희,국가혁명배당금당 김희전,NaN,NaN,NaN,NaN,NaN,NaN,제21대 국회의원선거 국회의원선거 서울특별시 용산구
3,국회의원선거,서울특별시,성동구,중구성동구갑,더불어민주당 홍익표,미래통합당 진수희,정의당 정혜연,국가혁명배당금당 이정섭,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제21대 국회의원선거 국회의원선거 서울특별시 성동구
4,국회의원선거,서울특별시,광진구,광진구갑,더불어민주당 전혜숙,미래통합당 김병민,민생당 임동순,정의당 오봉석,국가혁명배당금당 이승욱,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제21대 국회의원선거 국회의원선거 서울특별시 광진구


#### 21대 총선 양당 데이터만 집계 (df_21_candi_big2)

In [23]:
df_21_candi_big2 = df_21_candi.iloc[:, :6]
df_21_candi_big2

,선거구분,시도명,구시군명,선거구명,후보자1,후보자2
0,국회의원선거,서울특별시,종로구,종로구,더불어민주당 이낙연,미래통합당 황교안
1,국회의원선거,서울특별시,중구,중구성동구을,더불어민주당 박성준,미래통합당 지상욱
2,국회의원선거,서울특별시,용산구,용산구,더불어민주당 강태웅,미래통합당 권영세
3,국회의원선거,서울특별시,성동구,중구성동구갑,더불어민주당 홍익표,미래통합당 진수희
4,국회의원선거,서울특별시,광진구,광진구갑,더불어민주당 전혜숙,미래통합당 김병민
...,...,...,...,...,...,...
324,국회의원선거,경상남도,거창군,산청군함양군거창군합천군,더불어민주당 서필상,미래통합당 강석진
325,국회의원선거,경상남도,합천군,산청군함양군거창군합천군,더불어민주당 서필상,미래통합당 강석진
326,국회의원선거,제주특별자치도,제주시,제주시갑,더불어민주당 송재호,미래통합당 장성철
327,국회의원선거,제주특별자치도,제주시,제주시을,더불어민주당 오영훈,미래통합당 부상일


### 21대 총선 득표데이터 분리 (df_21_vote)

In [24]:
df_21_vote = df_21[~df_21['후보자별 득표수'].str.contains('더불어민주당')]
df_21_vote.shape

(12429, 23)

In [25]:
df_21_vote.head()

,선거구분,시도명,구시군명,선거구명,읍면동명,구분,선거인수,투표수,후보자별 득표수,후보자별 득표수.1,...,후보자별 득표수.5,후보자별 득표수.6,후보자별 득표수.7,후보자별 득표수.8,후보자별 득표수.9,후보자별 득표수.10,후보자별 득표수.11,무효투표수,기권수,조회결과
1,국회의원선거,서울특별시,종로구,종로구,합계,NaN,134516,95239,54902,37594,...,57,194,65,63,71,260,94038,1201,39277,제21대 국회의원선거 국회의원선거 서울특별시 종로구
2,국회의원선거,서울특별시,종로구,종로구,거소·선상투표,NaN,251,237,135,81,...,0,5,0,0,0,3,226,11,14,제21대 국회의원선거 국회의원선거 서울특별시 종로구
3,국회의원선거,서울특별시,종로구,종로구,관외사전투표,NaN,10007,9994,6472,3169,...,7,25,7,10,11,42,9823,171,13,제21대 국회의원선거 국회의원선거 서울특별시 종로구
4,국회의원선거,서울특별시,종로구,종로구,국외부재자투표,NaN,1177,321,232,80,...,1,0,0,0,1,2,317,4,856,제21대 국회의원선거 국회의원선거 서울특별시 종로구
5,국회의원선거,서울특별시,종로구,종로구,국외부재자투표(공관),NaN,0,12,11,1,...,0,0,0,0,0,0,12,0,-12,제21대 국회의원선거 국회의원선거 서울특별시 종로구


In [26]:
df_21_vote.dtypes

선거구분           object
시도명            object
구시군명           object
선거구명           object
읍면동명           object
구분             object
선거인수           object
투표수            object
후보자별 득표수       object
후보자별 득표수.1     object
후보자별 득표수.2     object
후보자별 득표수.3     object
후보자별 득표수.4     object
후보자별 득표수.5     object
후보자별 득표수.6     object
후보자별 득표수.7     object
후보자별 득표수.8     object
후보자별 득표수.9     object
후보자별 득표수.10    object
후보자별 득표수.11    object
무효투표수          object
기권수            object
조회결과           object
dtype: object

In [27]:
df_21_vote.columns = ['선거구분', '시도명', '구시군명', '선거구명', '읍면동명', '구분', '선거인수', '투표수',
                       '득표수1', '득표수2', '득표수3', '득표수4', '득표수5', '득표수6', '득표수7', '득표수8',
                       '득표수9', '득표수10', '득표수11', '득표수12', '무효투표수', '기권수', '조회결과']

In [28]:
df_21_vote.isnull().sum()

선거구분         0
시도명          0
구시군명         0
선거구명         0
읍면동명         0
구분        1974
선거인수         0
투표수          0
득표수1         0
득표수2         0
득표수3         0
득표수4       222
득표수5      2472
득표수6      6963
득표수7      9738
득표수8     11436
득표수9     12204
득표수10    12372
득표수11    12372
득표수12    12372
무효투표수        0
기권수          0
조회결과         0
dtype: int64

#### 21대 총선 선거구명 집계 (df_21_vote_sum)

In [29]:
df_21_vote_sum = df_21_vote[df_21_vote['읍면동명'] == '합계'].copy()
df_21_vote_sum

,선거구분,시도명,구시군명,선거구명,읍면동명,구분,선거인수,투표수,득표수1,득표수2,...,득표수6,득표수7,득표수8,득표수9,득표수10,득표수11,득표수12,무효투표수,기권수,조회결과
1,국회의원선거,서울특별시,종로구,종로구,합계,NaN,134516,95239,54902,37594,...,57,194,65,63,71,260,94038,1201,39277,제21대 국회의원선거 국회의원선거 서울특별시 종로구
59,국회의원선거,서울특별시,중구,중구성동구을,합계,NaN,183779,124700,64071,58300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1392,59079,제21대 국회의원선거 국회의원선거 서울특별시 중구
123,국회의원선거,서울특별시,용산구,용산구,합계,NaN,203233,135043,63001,63891,...,541,133643,NaN,NaN,NaN,NaN,NaN,1400,68190,제21대 국회의원선거 국회의원선거 서울특별시 용산구
178,국회의원선거,서울특별시,성동구,중구성동구갑,합계,NaN,192161,131200,70387,53107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1455,60961,제21대 국회의원선거 국회의원선거 서울특별시 성동구
225,국회의원선거,서울특별시,광진구,광진구갑,합계,NaN,156559,106506,56608,42822,...,105454,NaN,NaN,NaN,NaN,NaN,NaN,1052,50053,제21대 국회의원선거 국회의원선거 서울특별시 광진구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12577,국회의원선거,경상남도,거창군,산청군함양군거창군합천군,합계,NaN,53599,38976,6373,13147,...,390,17755,38307,NaN,NaN,NaN,NaN,669,14623,제21대 국회의원선거 국회의원선거 경상남도 거창군
12620,국회의원선거,경상남도,합천군,산청군함양군거창군합천군,합계,NaN,41197,29891,4063,12660,...,337,11597,29198,NaN,NaN,NaN,NaN,693,11306,제21대 국회의원선거 국회의원선거 경상남도 합천군
12679,국회의원선거,제주특별자치도,제주시,제주시갑,합계,NaN,208660,128031,61626,46909,...,6901,547,126540,NaN,NaN,NaN,NaN,1491,80629,제21대 국회의원선거 국회의원선거 제주특별자치도 제주시
12728,국회의원선거,제주특별자치도,제주시,제주시을,합계,NaN,191862,122883,67206,49862,...,0,0,121412,NaN,NaN,NaN,NaN,1471,68979,제21대 국회의원선거 국회의원선거 제주특별자치도 제주시


In [30]:
# NaN 옆의 값이 득표 총 합계값인데 행마다 출마인원이 달라 위치가 다름.
# 득표수12 컬럼까지 쭉 채워서 득표수12 컬럼을 득표총합으로 활용한다.
# 득표수1 / 득표수12 = 득표율1 로 처리
df_21_vote_sum = df_21_vote_sum.fillna(method = 'ffill', axis=1)

In [31]:
df_21_vote_sum = df_21_vote_sum.astype({
    '선거인수' : 'int',
    '투표수' : 'int',
    '득표수1' : 'int',
    '득표수2' : 'int',
    '득표수3' : 'int',
    '득표수4' : 'int',
    '득표수5' : 'int',
    '득표수6' : 'int',
    '득표수7' : 'int',
    '득표수8' : 'int',
    '득표수9' : 'int',
    '득표수10' : 'int',
    '득표수11' : 'int',
    '득표수12' : 'int',
    '무효투표수' : 'int',
    '기권수' : 'int'
})
df_21_vote_sum.dtypes

선거구분     object
시도명      object
구시군명     object
선거구명     object
읍면동명     object
구분       object
선거인수      int32
투표수       int32
득표수1      int32
득표수2      int32
득표수3      int32
득표수4      int32
득표수5      int32
득표수6      int32
득표수7      int32
득표수8      int32
득표수9      int32
득표수10     int32
득표수11     int32
득표수12     int32
무효투표수     int32
기권수       int32
조회결과     object
dtype: object

In [32]:
# 득표율 구하기
df_21_vote_sum['득표율1'] = df_21_vote_sum['득표수1'] / df_21_vote_sum['득표수12']
df_21_vote_sum['득표율2'] = df_21_vote_sum['득표수2'] / df_21_vote_sum['득표수12']
df_21_vote_sum

,선거구분,시도명,구시군명,선거구명,읍면동명,구분,선거인수,투표수,득표수1,득표수2,...,득표수8,득표수9,득표수10,득표수11,득표수12,무효투표수,기권수,조회결과,득표율1,득표율2
1,국회의원선거,서울특별시,종로구,종로구,합계,합계,134516,95239,54902,37594,...,65,63,71,260,94038,1201,39277,제21대 국회의원선거 국회의원선거 서울특별시 종로구,0.583828,0.399775
59,국회의원선거,서울특별시,중구,중구성동구을,합계,합계,183779,124700,64071,58300,...,123308,123308,123308,123308,123308,1392,59079,제21대 국회의원선거 국회의원선거 서울특별시 중구,0.519601,0.472800
123,국회의원선거,서울특별시,용산구,용산구,합계,합계,203233,135043,63001,63891,...,133643,133643,133643,133643,133643,1400,68190,제21대 국회의원선거 국회의원선거 서울특별시 용산구,0.471413,0.478072
178,국회의원선거,서울특별시,성동구,중구성동구갑,합계,합계,192161,131200,70387,53107,...,129745,129745,129745,129745,129745,1455,60961,제21대 국회의원선거 국회의원선거 서울특별시 성동구,0.542503,0.409318
225,국회의원선거,서울특별시,광진구,광진구갑,합계,합계,156559,106506,56608,42822,...,105454,105454,105454,105454,105454,1052,50053,제21대 국회의원선거 국회의원선거 서울특별시 광진구,0.536803,0.406073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12577,국회의원선거,경상남도,거창군,산청군함양군거창군합천군,합계,합계,53599,38976,6373,13147,...,38307,38307,38307,38307,38307,669,14623,제21대 국회의원선거 국회의원선거 경상남도 거창군,0.166366,0.343201
12620,국회의원선거,경상남도,합천군,산청군함양군거창군합천군,합계,합계,41197,29891,4063,12660,...,29198,29198,29198,29198,29198,693,11306,제21대 국회의원선거 국회의원선거 경상남도 합천군,0.139153,0.433591
12679,국회의원선거,제주특별자치도,제주시,제주시갑,합계,합계,208660,128031,61626,46909,...,126540,126540,126540,126540,126540,1491,80629,제21대 국회의원선거 국회의원선거 제주특별자치도 제주시,0.487008,0.370705
12728,국회의원선거,제주특별자치도,제주시,제주시을,합계,합계,191862,122883,67206,49862,...,121412,121412,121412,121412,121412,1471,68979,제21대 국회의원선거 국회의원선거 제주특별자치도 제주시,0.553537,0.410684


In [33]:
df_21_vote_sum['조회결과'].value_counts()

제21대 국회의원선거 국회의원선거 경기도 부천시         4
제21대 국회의원선거 국회의원선거 서울특별시 송파구       3
제21대 국회의원선거 국회의원선거 서울특별시 강남구       3
제21대 국회의원선거 국회의원선거 대구광역시 달서구       3
제21대 국회의원선거 국회의원선거 경기도 화성시         3
                                  ..
제21대 국회의원선거 국회의원선거 강원도 속초시         1
제21대 국회의원선거 국회의원선거 서울특별시 중구        1
제21대 국회의원선거 국회의원선거 강원도 양양군         1
제21대 국회의원선거 국회의원선거 강원도 인제군         1
제21대 국회의원선거 국회의원선거 제주특별자치도 서귀포시    1
Name: 조회결과, Length: 250, dtype: int64

## 21대 총선 최종데이터 (df_21_total) - 선거구 기준

In [34]:
df_21_total = pd.merge(df_21_candi_big2, df_21_vote_sum.loc[:, ['선거구분', '시도명', '구시군명', '선거구명', '득표율1', '득표율2']], how='left').reset_index(drop=True)
df_21_total

,선거구분,시도명,구시군명,선거구명,후보자1,후보자2,득표율1,득표율2
0,국회의원선거,서울특별시,종로구,종로구,더불어민주당 이낙연,미래통합당 황교안,0.583828,0.399775
1,국회의원선거,서울특별시,중구,중구성동구을,더불어민주당 박성준,미래통합당 지상욱,0.519601,0.472800
2,국회의원선거,서울특별시,용산구,용산구,더불어민주당 강태웅,미래통합당 권영세,0.471413,0.478072
3,국회의원선거,서울특별시,성동구,중구성동구갑,더불어민주당 홍익표,미래통합당 진수희,0.542503,0.409318
4,국회의원선거,서울특별시,광진구,광진구갑,더불어민주당 전혜숙,미래통합당 김병민,0.536803,0.406073
...,...,...,...,...,...,...,...,...
324,국회의원선거,경상남도,거창군,산청군함양군거창군합천군,더불어민주당 서필상,미래통합당 강석진,0.166366,0.343201
325,국회의원선거,경상남도,합천군,산청군함양군거창군합천군,더불어민주당 서필상,미래통합당 강석진,0.139153,0.433591
326,국회의원선거,제주특별자치도,제주시,제주시갑,더불어민주당 송재호,미래통합당 장성철,0.487008,0.370705
327,국회의원선거,제주특별자치도,제주시,제주시을,더불어민주당 오영훈,미래통합당 부상일,0.553537,0.410684


In [35]:
# XlsxWriter를 엔진으로 사용하여 Pandas Excel Writer를 작성
writer = pd.ExcelWriter('21대총선.xlsx', engine='xlsxwriter')

# 데이터 프레임을 각각 워크시트에 작성
df_21.to_excel(writer, sheet_name='21대_총선', index=False)

df_21_candi.to_excel(writer, sheet_name='21대_총선_후보자', index=False)
df_21_candi_big2.to_excel(writer, sheet_name='21대_총선_후보자_양당', index=False)

df_21_vote.to_excel(writer, sheet_name='21대_총선_득표', index=False)
df_21_vote_sum.to_excel(writer, sheet_name='21대_총선_득표_선거구명', index=False)

df_21_total.to_excel(writer, sheet_name='21대_총선_후보자득표', index=False)

# Pandas Excel Writer를 닫고 Excel 파일 저장
writer.save()

In [36]:
driver.quit()